1)-importing utile libraries and creating spark session 

In [22]:
import pandas as pd
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
###importing sparkContext
from pyspark import SparkContext
###Creating spark session 
from pyspark.sql import SparkSession        
###initializing SparkSession
spark=SparkSession.builder.master("local[3]").appName("my_Kmeans_Clustering").getOrCreate()
import configparser
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import date, timedelta, datetime, time

2.1)-Creating  "properties.conf" containing our program information

In [23]:
f=open('properties.conf', 'w')
f.write("[Bristol-City-bike]")
f.write("\nInput-data=data/Bristol-city-bike.json" )
f.write("\nOutput-data =exported/" )
f.write("\nKmeans-level= 3" )
f.close() 

2.2)-Utilizing "properties.conf" file to retrieve paths 

In [24]:
import configparser
config = configparser.ConfigParser()
config.read("properties.conf")
input_path=config['Bristol-City-bike']['Input-data']
output_path= config['Bristol-City-bike']['Output-data']
kmeans_partition = config['Bristol-City-bike']['Kmeans-level']

3)-Importing 'Bristol-City-bike.json' file using the output path already created 

In [25]:
bristol = spark.read.json(input_path)
bristol.show(2)

+--------------------+----------+----------+--------------------+------+
|             address|  latitude| longitude|                name|number|
+--------------------+----------+----------+--------------------+------+
|Lower River Tce /...|-27.482279|153.028723|122 - LOWER RIVER...|   122|
|Main St / Darragh St| -27.47059|153.036046|91 - MAIN ST / DA...|    91|
+--------------------+----------+----------+--------------------+------+
only showing top 2 rows



4)-Creating a new data frame "Kmeans-df", consiting of "latitude" and "longitude" only


In [17]:
Kmeans_df=bristol.select("latitude","longitude")
Kmeans_df.show(2)

+----------+----------+
|  latitude| longitude|
+----------+----------+
|-27.482279|153.028723|
| -27.47059|153.036046|
+----------+----------+
only showing top 2 rows



5)-Applying the k-means ML method on "Kmeans_df" data

In [26]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
features = ("longitude","latitude")
kmeans = KMeans().setK(int(kmeans_partition)).setSeed(1)
assembler = VectorAssembler(inputCols=features,outputCol="features")
dataset=assembler.transform(Kmeans_df)
model = kmeans.fit(dataset)
fitted = model.transform(dataset)
fitted.show(2)

+----------+----------+--------------------+----------+
|  latitude| longitude|            features|prediction|
+----------+----------+--------------------+----------+
|-27.482279|153.028723|[153.028723,-27.4...|         2|
| -27.47059|153.036046|[153.036046,-27.4...|         2|
+----------+----------+--------------------+----------+
only showing top 2 rows



6)-Verifying "fitted" data olumn names

In [27]:
fitted.columns

['latitude', 'longitude', 'features', 'prediction']

7.1)-determining the variables "longitude" and "latitude averages per cluster, using Spark DSL.

In [29]:
mean_DSL=fitted.groupBy("prediction").mean("latitude","longitude").show(5)
##Making sure we have 3 clusters
fitted.select(approx_count_distinct("prediction")).collect()

+----------+-------------------+------------------+
|prediction|      avg(latitude)|    avg(longitude)|
+----------+-------------------+------------------+
|         1|-27.460240636363633|153.04186302272726|
|         2| -27.47255990624999|   153.02594553125|
|         0|-27.481218536585374|153.00572882926832|
+----------+-------------------+------------------+



[Row(approx_count_distinct(prediction)=3)]

7.2)-determining the variables "longitude" and "latitude averages per cluster, using Spark DSL.

In [30]:
fitted.groupby("prediction").agg({"longitude":"avg", "latitude" : "avg"}).show()
###les 2 methodes SPark sql et spark DSL generent les memes resultats

+----------+-------------------+------------------+
|prediction|      avg(latitude)|    avg(longitude)|
+----------+-------------------+------------------+
|         1|-27.460240636363633|153.04186302272726|
|         2| -27.47255990624999|   153.02594553125|
|         0|-27.481218536585374|153.00572882926832|
+----------+-------------------+------------------+



9)-exporting fitted data frame after droping "features variable"

In [31]:
fitted=fitted.drop("features")
fitted.show(2)
fitted.write.csv('output_path/resultats.csv')

+----------+----------+----------+
|  latitude| longitude|prediction|
+----------+----------+----------+
|-27.482279|153.028723|         2|
| -27.47059|153.036046|         2|
+----------+----------+----------+
only showing top 2 rows

